<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [2]:
F = 1.
zF = 0.5
D = 0.55
B = F-D
qF = 1.

NT = 6
NF = 3

tray_m = 1.
cond_m = 5.
reboil_m = 5.

R = 2.

alpha = 1.5

x0 = np.full(NT+2,zF)

m0 = np.full(NT+2,tray_m)
L0 = np.full(NT+2, R*D)
L0[-1] = R*D
L0[0] = B
L0[:NF+1] += qF*F

m0[0] = reboil_m
m0[-1] = cond_m


weir_const = 0.1

m0[1:-1] = (L0[1:-1]/weir_const)**(2/3) + tray_m

tend = 500

In [18]:
def rhs(t, vec, R):
    x, m = np.split(vec, 2)

    y = alpha*x/(1+(alpha-1)*x)
    y[-1]=x[-1]

    V = np.full(NT+2,(R+1)*D)
    V[-1]=0
    V[:NF]-= (1-qF)*F

    L=np.zeros_like(m)
    L[-1] = R*D
    L[0] = B + 10*(m[0]-reboil_m)

    m_excess = m[1:-1] - tray_m
    L[1:-1] = np.where(m_excess>0, weir_const*(m_excess)**(3/2), 0.)

    dm=np.zeros_like(m)
    dmx=np.zeros_like(m)

    dm[1:-1] = L[2:] - L[1:-1] + V[:-2] - V[1:-1]
    dmx[1:-1] = x[2:]*L[2:] - x[1:-1]*L[1:-1] + y[:-2]*V[:-2] - y[1:-1]*V[1:-1]

    dm[NF] += F
    dmx[NF] += zF*F

    dm[0] = L[1] - L[0] - V[0]
    dmx[0] = x[1]*L[1] - x[0]*L[0] - y[0]*V[0]

    dm[-1] = V[-2] - (R+1)*D
    dmx[-1] = y[-2]*V[-2] - x[-1]*(R+1)*D

    dx = (dmx - x*dm)/m
    return np.concatenate([dx,dm])


In [19]:
res=solve_ivp(rhs, (0,tend), np.concatenate([x0,m0]), method='Radau', dense_output=True, args=(2.,))

In [20]:
tplot=np.linspace(0,tend,100)
xplots, mplots = np.split(res.sol(tplot),2,axis=0)
fig=make_subplots(rows=1,cols=2)

for i in range(NT+2):
    fig.add_scatter(x=tplot, y=xplots[i], name=f'x{i}', row=1, col=1)
    fig.add_scatter(x=tplot, y=mplots[i], name=f'm{i}', row=1, col=2)

fig.update_layout(width=1000, height=500,template='plotly_dark')

In [22]:
res2=solve_ivp(rhs, (0,tend), res.y[:,-1], method='Radau', dense_output=True, args=(2.5,))
xplots, mplots = np.split(res2.sol(tplot),2,axis=0)
fig=make_subplots(rows=1,cols=2)

for i in range(NT+2):
    fig.add_scatter(x=tplot, y=xplots[i], name=f'x{i}', row=1, col=1)
    fig.add_scatter(x=tplot, y=mplots[i], name=f'm{i}', row=1, col=2)

fig.update_layout(width=1000, height=500,template='plotly_dark')